In [5]:
# Example from Parkfield 2004 

"""Tutorial to illustrate the lag_calc usage."""
import logging
from multiprocessing import cpu_count

from obspy.clients.fdsn import Client
from obspy.core.event import Catalog
from obspy import UTCDateTime

from eqcorrscan.core import template_gen, match_filter, lag_calc
from eqcorrscan.utils import pre_processing, catalog_utils

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s\t%(name)s\t%(levelname)s\t%(message)s")


#def run_tutorial(min_magnitude=2, shift_len=0.2, num_cores=4, min_cc=0.5):
min_magnitude = 2
shift_len = 0.2
num_cores = 4
min_cc = 0.5

"""Functional, tested example script for running the lag-calc tutorial."""
if num_cores > cpu_count():
    num_cores = cpu_count()
client = Client('NCEDC')
t1 = UTCDateTime(2004, 9, 28)
t2 = t1 + 86400
print('Downloading catalog')
catalog = client.get_events(
    starttime=t1, endtime=t2, minmagnitude=min_magnitude,
    minlatitude=35.7, maxlatitude=36.1, minlongitude=-120.6,
    maxlongitude=-120.2, includearrivals=True)
# We don't need all the picks, lets take the information from the
# five most used stations - note that this is done to reduce computational
# costs.
catalog = catalog_utils.filter_picks(
    catalog, channels=['EHZ'], top_n_picks=5)
# There is a duplicate pick in event 3 in the catalog - this has the effect
# of reducing our detections - check it yourself.
for pick in catalog[3].picks:
    if pick.waveform_id.station_code == 'PHOB' and \
                    pick.onset == 'emergent':
        catalog[3].picks.remove(pick)
print('Generating templates')
templates = template_gen.template_gen(
    method="from_client", catalog=catalog, client_id='NCEDC',
    lowcut=2.0, highcut=9.0, samp_rate=50.0, filt_order=4, length=3.0,
    prepick=0.15, swin='all', process_len=3600)
# In this section we generate a series of chunks of data.
start_time = UTCDateTime(2004, 9, 28, 17)
end_time = UTCDateTime(2004, 9, 28, 20)
process_len = 3600
chunks = []
chunk_start = start_time
while chunk_start < end_time:
    chunk_end = chunk_start + process_len
    if chunk_end > end_time:
        chunk_end = end_time
    chunks.append((chunk_start, chunk_end))
    chunk_start += process_len

all_detections = []
picked_catalog = Catalog()
template_names = [template[0].stats.starttime.strftime("%Y%m%d_%H%M%S")
                  for template in templates]
for t1, t2 in chunks:
    print('Downloading and processing for start-time: %s' % t1)
    # Download and process the data
    bulk_info = [(tr.stats.network, tr.stats.station, '*',
                  tr.stats.channel, t1, t2) for tr in templates[0]]
    # Just downloading a chunk of data
    st = client.get_waveforms_bulk(bulk_info)
    st.merge(fill_value='interpolate')
    st = pre_processing.shortproc(
        st, lowcut=2.0, highcut=9.0, filt_order=4, samp_rate=50.0,
        num_cores=num_cores)
    detections = match_filter.match_filter(
        template_names=template_names, template_list=templates, st=st,
        threshold=8.0, threshold_type='MAD', trig_int=6.0, plotvar=False,
        plotdir='.', cores=num_cores)
    # Extract unique detections from set.
    unique_detections = []
    for master in detections:
        keep = True
        for slave in detections:
            if not master == slave and\
               abs(master.detect_time - slave.detect_time) <= 1.0:
                # If the events are within 1s of each other then test which
                # was the 'best' match, strongest detection
                if not master.detect_val > slave.detect_val:
                    keep = False
                    break
        if keep:
            unique_detections.append(master)
    all_detections += unique_detections

    picked_catalog += lag_calc.lag_calc(
        detections=unique_detections, detect_data=st,
        template_names=template_names, templates=templates,
        shift_len=shift_len, min_cc=min_cc, interpolate=False, plot=False)
# Return all of this so that we can use this function for testing.
return all_detections, picked_catalog, templates, template_names


#if __name__ == '__main__':
#    run_tutorial(min_magnitude=4, num_cores=cpu_count())

2020-11-08 16:20:22,912	eqcorrscan.core.template_gen	INFO	Downloading data
2020-11-08 16:20:22,914	eqcorrscan.core.template_gen	INFO	Downloading for start-time: 2004-09-28T17:13:54.250000Z end-time: 2004-09-28T18:13:54.250000Z


Generating templates


2020-11-08 16:20:24,128	eqcorrscan.core.template_gen	INFO	Downloading for start-time: 2004-09-28T17:13:54.250000Z end-time: 2004-09-28T18:13:54.250000Z
2020-11-08 16:20:25,819	eqcorrscan.core.template_gen	INFO	Downloading for start-time: 2004-09-28T17:13:54.250000Z end-time: 2004-09-28T18:13:54.250000Z
2020-11-08 16:20:28,425	eqcorrscan.core.template_gen	INFO	Downloading for start-time: 2004-09-28T17:13:54.250000Z end-time: 2004-09-28T18:13:54.250000Z
2020-11-08 16:20:29,694	eqcorrscan.core.template_gen	INFO	Downloading for start-time: 2004-09-28T17:13:54.250000Z end-time: 2004-09-28T18:13:54.250000Z
2020-11-08 16:20:30,993	eqcorrscan.core.template_gen	INFO	Pre-processing data
2020-11-08 16:20:31,204	eqcorrscan.core.template_gen	INFO	Working on channel PCA.EHZ
2020-11-08 16:20:31,205	eqcorrscan.core.template_gen	INFO	Found Trace NC.PCA..EHZ | 2004-09-28T17:13:54.250000Z - 2004-09-28T18:13:54.230000Z | 50.0 Hz, 180000 samples
2020-11-08 16:20:31,206	eqcorrscan.core.template_gen	WARNING	

2020-11-08 16:20:31,292	eqcorrscan.core.template_gen	INFO	Working on channel PMR.EHZ
2020-11-08 16:20:31,293	eqcorrscan.core.template_gen	INFO	Found Trace NC.PMR..EHZ | 2004-09-28T17:13:54.250000Z - 2004-09-28T18:13:54.230000Z | 50.0 Hz, 180000 samples
2020-11-08 16:20:31,294	eqcorrscan.core.template_gen	WARNING	Pick for PMR.EHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
2020-11-08 16:20:31,296	eqcorrscan.core.template_gen	INFO	Working on channel PSR.EHZ
2020-11-08 16:20:31,297	eqcorrscan.core.template_gen	INFO	Found Trace NC.PSR..EHZ | 2004-09-28T17:13:54.250000Z - 2004-09-28T18:13:54.230000Z | 50.0 Hz, 180000 samples
2020-11-08 16:20:31,298	eqcorrscan.core.template_gen	WARNING	Pick for PSR.EHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
2020-11-08 16:20:31,299	eqcorrscan.core.template_gen	INFO	Working on channel PST.EHZ
2020-11-08 16:20:31,300	eqcorrscan.core.template_gen	INFO	Found Trace

2020-11-08 16:20:31,385	eqcorrscan.core.template_gen	WARNING	Pick for PHF.EHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
2020-11-08 16:20:31,386	eqcorrscan.core.template_gen	INFO	Working on channel PMR.EHZ
2020-11-08 16:20:31,387	eqcorrscan.core.template_gen	INFO	Found Trace NC.PMR..EHZ | 2004-09-28T17:13:54.250000Z - 2004-09-28T18:13:54.230000Z | 50.0 Hz, 180000 samples
2020-11-08 16:20:31,387	eqcorrscan.core.template_gen	WARNING	Pick for PMR.EHZ has no phase hint given, you should not use this template for cross-correlation re-picking!
2020-11-08 16:20:31,388	eqcorrscan.core.template_gen	INFO	Working on channel PSR.EHZ
2020-11-08 16:20:31,389	eqcorrscan.core.template_gen	INFO	Found Trace NC.PSR..EHZ | 2004-09-28T17:13:54.250000Z - 2004-09-28T18:13:54.230000Z | 50.0 Hz, 180000 samples
2020-11-08 16:20:31,390	eqcorrscan.core.template_gen	WARNING	Pick for PSR.EHZ has no phase hint given, you should not use this template for cross-correlat

...

2020-11-08 16:21:33,508	eqcorrscan.core.match_filter.matched_filter	WARNING	plotvar is depreciated, use plot instead
2020-11-08 16:21:33,509	eqcorrscan.core.match_filter.matched_filter	INFO	Checking for spikes in data
2020-11-08 16:21:33,577	eqcorrscan.core.match_filter.matched_filter	INFO	Copying data to keep your input safe
2020-11-08 16:21:33,616	eqcorrscan.core.match_filter.matched_filter	INFO	Reshaping templates
2020-11-08 16:21:33,642	eqcorrscan.core.match_filter.matched_filter	INFO	Starting the correlation run for these data
2020-11-08 16:21:34,536	eqcorrscan.core.match_filter.matched_filter	INFO	Looping over templates and streams took: 0.8795s
2020-11-08 16:21:34,772	eqcorrscan.core.match_filter.matched_filter	INFO	Finding peaks took 0.0954s
2020-11-08 16:21:34,951	eqcorrscan.core.match_filter.matched_filter	INFO	Made 176 detections from 74 templates
2020-11-08 16:21:35,023	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T17:15:27.710000Z - 2

2020-11-08 16:21:35,588	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T19:26:09.080000Z - 2004-09-28T19:26:12.060000Z | 50.0 Hz, 150 samples
2020-11-08 16:21:35,695	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T19:31:31.040000Z - 2004-09-28T19:31:34.020000Z | 50.0 Hz, 150 samples
2020-11-08 16:21:35,696	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T19:34:00.890000Z - 2004-09-28T19:34:03.870000Z | 50.0 Hz, 150 samples
2020-11-08 16:21:35,716	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T19:40:27.570000Z - 2004-09-28T19:40:30.550000Z | 50.0 Hz, 150 samples
2020-11-08 16:21:35,731	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T19:48:25.790000Z - 2004-09-28T19:48:28.770000Z | 50.0 Hz, 150 samples
2020-11-08 16:21:35,745	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09

2020-11-08 16:21:40,645	eqcorrscan.core.match_filter.matched_filter	WARNING	plotvar is depreciated, use plot instead
2020-11-08 16:21:40,646	eqcorrscan.core.match_filter.matched_filter	INFO	Checking for spikes in data
2020-11-08 16:21:40,713	eqcorrscan.core.match_filter.matched_filter	INFO	Copying data to keep your input safe
2020-11-08 16:21:40,754	eqcorrscan.core.match_filter.matched_filter	INFO	Reshaping templates
2020-11-08 16:21:40,778	eqcorrscan.core.match_filter.matched_filter	INFO	Starting the correlation run for these data
2020-11-08 16:21:41,658	eqcorrscan.core.match_filter.matched_filter	INFO	Looping over templates and streams took: 0.8661s
2020-11-08 16:21:41,891	eqcorrscan.core.match_filter.matched_filter	INFO	Finding peaks took 0.0976s
2020-11-08 16:21:42,045	eqcorrscan.core.match_filter.matched_filter	INFO	Made 149 detections from 74 templates
2020-11-08 16:21:42,104	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T17:15:27.710000Z - 2

2020-11-08 16:21:42,503	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T19:26:09.080000Z - 2004-09-28T19:26:12.060000Z | 50.0 Hz, 150 samples
2020-11-08 16:21:42,574	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T19:31:31.040000Z - 2004-09-28T19:31:34.020000Z | 50.0 Hz, 150 samples
2020-11-08 16:21:42,588	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T19:34:00.890000Z - 2004-09-28T19:34:03.870000Z | 50.0 Hz, 150 samples
2020-11-08 16:21:42,628	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T19:40:27.570000Z - 2004-09-28T19:40:30.550000Z | 50.0 Hz, 150 samples
2020-11-08 16:21:42,642	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09-28T19:48:25.790000Z - 2004-09-28T19:48:28.770000Z | 50.0 Hz, 150 samples
2020-11-08 16:21:42,660	eqcorrscan.core.lag_calc	INFO	Running lag-calc for template NC.PCA..EHZ | 2004-09

SyntaxError: 'return' outside function (<ipython-input-5-945cd6982046>, line 106)

In [ ]:
catalog
templates
st 
all_detections

picked_catalog

In [12]:
st

5 Trace(s) in Stream:
NC.PCA..EHZ | 2004-09-28T19:00:00.000000Z - 2004-09-28T19:59:59.980000Z | 50.0 Hz, 180000 samples
NC.PHF..EHZ | 2004-09-28T19:00:00.000000Z - 2004-09-28T19:59:59.980000Z | 50.0 Hz, 180000 samples
NC.PMR..EHZ | 2004-09-28T19:00:00.000000Z - 2004-09-28T19:59:59.980000Z | 50.0 Hz, 180000 samples
NC.PSR..EHZ | 2004-09-28T19:00:00.000000Z - 2004-09-28T19:59:59.980000Z | 50.0 Hz, 180000 samples
NC.PST..EHZ | 2004-09-28T19:00:00.000000Z - 2004-09-28T19:59:59.980000Z | 50.0 Hz, 180000 samples